# Assignment 3 — COMP 511

**Q1**: Node Classification — Label Propagation · Node2Vec · GCN  
**Q2**: Link Prediction — Common Neighbours · Jaccard · GCN encoder-decoder  

Datasets: Cora · CiteSeer · PubMed · ogbn-proteins  
All results: mean ± std over 10 independent runs.

In [ ]:
# ── Imports ───────────────────────────────────────────────────────────────────
import os, warnings
import numpy as np
import scipy.sparse as sp
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings('ignore')

from node2vec import Node2Vec as GensimNode2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.data import Data as PyGData

DEVICE = torch.device('mps' if torch.backends.mps.is_available() else
                       'cuda' if torch.cuda.is_available() else 'cpu')
print(f'Accelerator: {DEVICE}')

N_RUNS = 10

def fix_seed(s):
    torch.manual_seed(s)
    np.random.seed(s)

In [ ]:
# ── Dataset Loading ───────────────────────────────────────────────────────────
# Planetoid datasets auto-downloaded by torch_geometric (same automated
# approach as A2's gdown downloads). Extracted as scipy/numpy arrays
# to match the format expected by our custom LP and GCN implementations.

pyg_catalog = {}
for tag in ['Cora', 'CiteSeer', 'PubMed']:
    pyg_catalog[tag.lower()] = Planetoid(root=f'data/{tag}', name=tag,
                                          transform=T.NormalizeFeatures())
    d = pyg_catalog[tag.lower()][0]
    print(f"{tag:10s}: {d.num_nodes:6d} nodes  {d.num_edges:6d} edges  "
          f"{pyg_catalog[tag.lower()].num_features:4d} feats  "
          f"{pyg_catalog[tag.lower()].num_classes} classes  "
          f"tr={d.train_mask.sum().item()} va={d.val_mask.sum().item()} te={d.test_mask.sum().item()}")

# ogbn-proteins: auto-downloaded by OGB
prot_ds     = PygNodePropPredDataset(name='ogbn-proteins', root='data/ogbn-proteins')
prot_splits = prot_ds.get_idx_split()
prot_data   = prot_ds[0]

# 8-dim node features: mean of incident edge attributes (OGB standard approach)
_ei  = prot_data.edge_index
_ea  = prot_data.edge_attr.float()
_x   = torch.zeros(prot_data.num_nodes, _ea.size(1))
_cnt = torch.zeros(prot_data.num_nodes, 1)
_x.scatter_add_(0, _ei[0].unsqueeze(1).expand_as(_ea), _ea)
_cnt.scatter_add_(0, _ei[0].unsqueeze(1), torch.ones(_ei.size(1), 1))
_cnt[_cnt == 0] = 1
prot_data.x = _x / _cnt

print(f"\nogbn-proteins: {prot_data.num_nodes:,} nodes  {prot_data.num_edges:,} edges  "
      f"x={tuple(prot_data.x.shape)}  y={tuple(prot_data.y.shape)}")
print(f"  tr={len(prot_splits['train']):,}  va={len(prot_splits['valid']):,}  "
      f"te={len(prot_splits['test']):,}")

In [ ]:
# ── Shared Utilities ──────────────────────────────────────────────────────────

def build_adj(edge_index: torch.Tensor, n_nodes: int) -> sp.csr_matrix:
    """Symmetric sparse adjacency from a PyG edge_index."""
    r, c  = edge_index.numpy()
    rows  = np.concatenate([r, c])
    cols  = np.concatenate([c, r])
    mat   = sp.coo_matrix((np.ones(len(rows)), (rows, cols)), shape=(n_nodes, n_nodes)).tocsr()
    mat.data = np.ones_like(mat.data)     # binarise duplicate entries
    return mat


def row_normalise(M: sp.spmatrix) -> sp.spmatrix:
    """Return D^{-1} A (row-stochastic form)."""
    sums  = np.array(M.sum(axis=1)).flatten()
    inv_d = np.where(sums > 0, 1.0 / sums, 0.0)
    return sp.diags(inv_d).dot(M)


def scipy_to_sparse_torch(M: sp.spmatrix) -> torch.Tensor:
    """Convert scipy sparse → torch sparse COO float tensor."""
    coo  = M.tocoo().astype(np.float32)
    idx  = torch.from_numpy(np.vstack([coo.row, coo.col]).astype(np.int64))
    vals = torch.from_numpy(coo.data)
    return torch.sparse_coo_tensor(idx, vals, torch.Size(coo.shape))


def extract_arrays(pyg_data, n_cls):
    """Pull a PyG Data object into scipy/numpy arrays (LP.py / gcn.py format)."""
    adjacency = build_adj(pyg_data.edge_index, pyg_data.num_nodes)
    feats     = pyg_data.x.numpy().astype(np.float32)
    y_int     = pyg_data.y.numpy().flatten()
    labels_oh = np.zeros((pyg_data.num_nodes, n_cls), dtype=np.float32)
    labels_oh[np.arange(pyg_data.num_nodes), y_int] = 1.0
    tr = pyg_data.train_mask.numpy().nonzero()[0].tolist()
    va = pyg_data.val_mask.numpy().nonzero()[0].tolist()
    te = pyg_data.test_mask.numpy().nonzero()[0].tolist()
    return adjacency, feats, labels_oh, tr, va, te

---## Q1 — Node Classification| Algorithm | Features | Notes ||-----------|----------|-------|| **Label Propagation** | none (structure only) | deterministic → std = 0 || **Node2Vec** | none (structure only) | gensim random-walk embeddings + LogReg || **GCN** | node features (required) | custom 2-layer implementation |

In [ ]:
# ── Label Propagation ─────────────────────────────────────────────────────────
# Based on LP.py structure: iterative diffusion Y ← α·S·Y + (1-α)·Y₀
# with hard clamping of labelled nodes after every step.
# No features used. Deterministic → all 10 runs identical, std = 0.

def diffuse_labels(transition, seed_mat, anchor_nodes,
                   decay=0.99, max_steps=1000, eps=1e-6):
    """
    Propagate label beliefs through the graph.
    transition  : row-normalised adjacency (scipy sparse)
    seed_mat    : (n, C) float, non-zero only at training rows
    anchor_nodes: indices re-clamped to seed_mat each iteration
    Returns     : converged belief matrix, per-step Frobenius residuals
    """
    belief    = seed_mat.copy()
    residuals = []
    for _ in range(max_steps):
        prev   = belief.copy()
        belief = decay * transition.dot(belief) + (1.0 - decay) * seed_mat
        belief[anchor_nodes, :] = seed_mat[anchor_nodes, :]   # clamp
        delta  = np.linalg.norm(belief - prev, ord='fro')
        residuals.append(delta)
        if delta < eps:
            break
    return belief, residuals


def run_lp(adj, labels_oh, tr_idx, va_idx, te_idx,
           decay=0.99, max_steps=1000, n_runs=N_RUNS):
    transition = row_normalise(adj)
    y_hard     = np.argmax(labels_oh, axis=1)
    seed_mat   = np.zeros_like(labels_oh)
    seed_mat[tr_idx, :] = labels_oh[tr_idx, :]

    beliefs, residuals = diffuse_labels(transition, seed_mat, tr_idx, decay, max_steps)
    preds     = np.argmax(beliefs, axis=1)
    test_acc  = float(np.mean(preds[te_idx] == y_hard[te_idx]))
    scores    = [test_acc] * n_runs
    print(f"  LP  acc={test_acc:.4f}  (deterministic, std=0)")
    return scores, residuals


def run_lp_proteins(data, splits, decay=0.85, max_steps=100, n_runs=N_RUNS):
    """Vectorised LP for ogbn-proteins (112 binary tasks, macro ROC-AUC)."""
    adj        = build_adj(data.edge_index, data.num_nodes)
    transition = row_normalise(adj)
    y          = data.y.numpy().astype(float)
    tr_idx     = splits['train'].numpy()
    te_idx     = splits['test'].numpy()

    seed_mat   = np.zeros_like(y)
    seed_mat[tr_idx] = y[tr_idx]
    beliefs, residuals = diffuse_labels(transition, seed_mat, tr_idx, decay, max_steps)

    def macro_auc(idx):
        yt = y[idx].astype(int);  yp = beliefs[idx]
        vc = (yt.sum(0) > 0) & (yt.sum(0) < len(idx))
        return roc_auc_score(yt[:, vc], yp[:, vc], average='macro') if vc.sum() else float('nan')

    auc    = macro_auc(te_idx)
    scores = [auc] * n_runs
    print(f"  LP  auc={auc:.4f}  (deterministic, std=0)")
    return scores, residuals

In [ ]:
# ── Node2Vec ──────────────────────────────────────────────────────────────────
# Based on node_to_vector.py: gensim-based random walks → LogisticRegression.
# No features used. Stochastic (different seeds each run) → non-zero variance.

def embed_graph(nx_graph, embed_dim=64, walk_len=40, n_walks=10,
                p=1, q=1, seed=0):
    """Train Node2Vec and return (n_nodes, embed_dim) embedding matrix."""
    walker   = GensimNode2Vec(nx_graph, dimensions=embed_dim,
                               walk_length=walk_len, num_walks=n_walks,
                               p=p, q=q, workers=1, seed=seed, quiet=True)
    wv_model = walker.fit(window=10, min_count=1, batch_words=4)
    node_ids = sorted(nx_graph.nodes())
    return np.array([wv_model.wv[str(nid)] for nid in node_ids])


def run_n2v_planetoid(data, n_cls, n_runs=N_RUNS, embed_dim=64, walk_len=40):
    adj, _, labels_oh, tr_idx, va_idx, te_idx = extract_arrays(data, n_cls)
    nx_graph    = nx.from_scipy_sparse_array(adj)
    true_labels = np.argmax(labels_oh, axis=1)
    test_accs   = []

    for run in range(n_runs):
        emb = embed_graph(nx_graph, embed_dim=embed_dim,
                          walk_len=walk_len, seed=run * 13)
        clf = LogisticRegression(max_iter=300, random_state=run, C=1.0)
        clf.fit(emb[tr_idx], true_labels[tr_idx])
        acc = accuracy_score(true_labels[te_idx], clf.predict(emb[te_idx]))
        test_accs.append(acc)
        print(f"  N2V run {run+1:02d}/{n_runs}: acc={acc:.4f}")
    return test_accs


def run_n2v_proteins(data, splits, n_runs=N_RUNS,
                     embed_dim=32, walk_len=10, n_walks=3):
    """Node2Vec for ogbn-proteins (multilabel ROC-AUC, reduced walk settings)."""
    adj      = build_adj(data.edge_index, data.num_nodes)
    nx_graph = nx.from_scipy_sparse_array(adj)
    y        = data.y.numpy().astype(int)
    tr_idx   = splits['train'].numpy()
    te_idx   = splits['test'].numpy()
    test_aucs = []

    for run in range(n_runs):
        emb = embed_graph(nx_graph, embed_dim=embed_dim,
                          walk_len=walk_len, n_walks=n_walks, seed=run * 13)
        clf = MultiOutputClassifier(
            LogisticRegression(max_iter=200, random_state=run, C=0.5), n_jobs=-1)
        clf.fit(emb[tr_idx], y[tr_idx])
        probs = np.stack([e.predict_proba(emb[te_idx])[:, 1]
                          for e in clf.estimators_], axis=1)
        yt    = y[te_idx]
        vc    = (yt.sum(0) > 0) & (yt.sum(0) < len(te_idx))
        auc   = roc_auc_score(yt[:, vc], probs[:, vc], average='macro')
        test_aucs.append(auc)
        print(f"  N2V proteins run {run+1:02d}/{n_runs}: auc={auc:.4f}")
    return test_aucs

In [ ]:
# ── Custom GCN ────────────────────────────────────────────────────────────────
# Based on gcn.py structure: custom layer using torch.spmm on pre-computed
# sparse normalised adjacency.  Node features are used (required for GCN).

class SpectralConvBlock(nn.Module):
    """Graph conv: dropout → sparse propagation → linear → ReLU."""
    def __init__(self, ch_in, ch_out, p_drop):
        super().__init__()
        self.proj   = nn.Linear(ch_in, ch_out)
        self.p_drop = p_drop

    def forward(self, h, norm_adj):
        h = F.dropout(h, p=self.p_drop, training=self.training)
        h = torch.spmm(norm_adj, h)
        return F.relu(self.proj(h))


class NodeClassifierNet(nn.Module):
    """2-layer GCN for multi-class node classification (gcn.py layout)."""
    def __init__(self, n_feat, n_hid, n_cls, p_drop=0.5):
        super().__init__()
        self.layer_in  = SpectralConvBlock(n_feat, n_hid, p_drop)
        self.layer_out = SpectralConvBlock(n_hid,  n_cls, p_drop)
        self.p_drop    = p_drop

    def forward(self, x, norm_adj):
        h = self.layer_in(x, norm_adj)
        h = F.dropout(h, p=self.p_drop, training=self.training)
        h = torch.spmm(norm_adj, h)
        return F.log_softmax(self.layer_out.proj(h), dim=1)


def train_one_gcn(adj, feats, labels_oh, tr_idx, va_idx, te_idx,
                  n_hid=16, epochs=200, lr=0.01, wd=5e-4,
                  p_drop=0.5, patience=10, seed=0):
    fix_seed(seed)
    norm_adj = row_normalise(adj)
    feat_t   = torch.FloatTensor(feats)
    lbl_t    = torch.LongTensor(np.argmax(labels_oh, axis=1))
    adj_t    = scipy_to_sparse_torch(norm_adj)
    tr_t, va_t, te_t = (torch.LongTensor(x) for x in (tr_idx, va_idx, te_idx))

    net   = NodeClassifierNet(feats.shape[1], n_hid, labels_oh.shape[1], p_drop)
    opt   = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
    best_val, pat_ctr, best_state = float('inf'), 0, None
    tr_hist, va_hist = [], []

    for _ in range(epochs):
        net.train();  opt.zero_grad()
        logits  = net(feat_t, adj_t)
        tr_loss = F.nll_loss(logits[tr_t], lbl_t[tr_t])
        tr_loss.backward();  opt.step()

        net.eval()
        with torch.no_grad():
            logits  = net(feat_t, adj_t)
            va_loss = F.nll_loss(logits[va_t], lbl_t[va_t]).item()
        tr_hist.append(tr_loss.item());  va_hist.append(va_loss)

        if va_loss < best_val:
            best_val  = va_loss
            best_state = {k: v.clone() for k, v in net.state_dict().items()}
            pat_ctr   = 0
        else:
            pat_ctr  += 1
        if pat_ctr >= patience:
            break

    net.load_state_dict(best_state);  net.eval()
    with torch.no_grad():
        pred = net(feat_t, adj_t)[te_t].argmax(1)
        acc  = (pred == lbl_t[te_t]).float().mean().item()
    return acc, tr_hist, va_hist


def run_gcn_planetoid(adj, feats, labels_oh, tr_idx, va_idx, te_idx,
                      n_runs=N_RUNS, **kw):
    test_accs, all_tr, all_va = [], [], []
    for run in range(n_runs):
        acc, tr_h, va_h = train_one_gcn(adj, feats, labels_oh,
                                         tr_idx, va_idx, te_idx,
                                         seed=run * 17, **kw)
        test_accs.append(acc);  all_tr.append(tr_h);  all_va.append(va_h)
        print(f"  GCN run {run+1:02d}/{n_runs}: acc={acc:.4f}")
    return test_accs, all_tr, all_va


class ProteinsGCN(nn.Module):
    """3-layer GCN with BatchNorm for ogbn-proteins multilabel task."""
    def __init__(self, n_feat, n_hid, n_cls=112, p_drop=0.5):
        super().__init__()
        self.c1  = nn.Linear(n_feat, n_hid);  self.bn1 = nn.BatchNorm1d(n_hid)
        self.c2  = nn.Linear(n_hid,  n_hid);  self.bn2 = nn.BatchNorm1d(n_hid)
        self.c3  = nn.Linear(n_hid,  n_cls);  self.p   = p_drop

    def forward(self, x, adj_sp):
        h = F.relu(self.bn1(self.c1(torch.spmm(adj_sp, x))))
        h = F.dropout(h, p=self.p, training=self.training)
        h = F.relu(self.bn2(self.c2(torch.spmm(adj_sp, h))))
        h = F.dropout(h, p=self.p, training=self.training)
        return self.c3(torch.spmm(adj_sp, h))


def run_gcn_proteins(data, splits, n_runs=N_RUNS, n_hid=256, epochs=100, lr=1e-2):
    adj    = build_adj(data.edge_index, data.num_nodes)
    adj_t  = scipy_to_sparse_torch(row_normalise(adj)).to(DEVICE)
    feat_t = data.x.to(DEVICE)
    y_t    = data.y.float().to(DEVICE)
    tr_idx = splits['train'].to(DEVICE)
    va_idx = splits['valid'].to(DEVICE)
    te_idx = splits['test']
    y_test = data.y[te_idx].numpy().astype(int)
    test_aucs, all_tr, all_va = [], [], []

    for run in range(n_runs):
        fix_seed(run * 17)
        net  = ProteinsGCN(feat_t.size(1), n_hid).to(DEVICE)
        opt  = torch.optim.Adam(net.parameters(), lr=lr)
        crit = nn.BCEWithLogitsLoss()
        tr_h, va_h = [], []
        for _ in range(epochs):
            net.train();  opt.zero_grad()
            out  = net(feat_t, adj_t)
            loss = crit(out[tr_idx], y_t[tr_idx])
            loss.backward();  opt.step()
            net.eval()
            with torch.no_grad():
                out    = net(feat_t, adj_t)
                va_l   = crit(out[va_idx], y_t[va_idx]).item()
            tr_h.append(loss.item());  va_h.append(va_l)
        net.eval()
        with torch.no_grad():
            probs = torch.sigmoid(net(feat_t, adj_t)[te_idx.to(DEVICE)]).cpu().numpy()
        vc  = (y_test.sum(0) > 0) & (y_test.sum(0) < len(y_test))
        auc = roc_auc_score(y_test[:, vc], probs[:, vc], average='macro')
        test_aucs.append(auc);  all_tr.append(tr_h);  all_va.append(va_h)
        print(f"  GCN proteins run {run+1:02d}/{n_runs}: auc={auc:.4f}")
    return test_aucs, all_tr, all_va

In [ ]:
# ── Q1 Experiments ────────────────────────────────────────────────────────────
q1_store = {}

for tag in ['cora', 'citeseer', 'pubmed']:
    ds    = pyg_catalog[tag]
    data  = ds[0]
    adj, feats, labels_oh, tr_idx, va_idx, te_idx = extract_arrays(data, ds.num_classes)
    print(f"\n{'='*56}\n{tag.upper()}  ({ds.num_classes} classes)\n{'='*56}")
    q1_store[tag] = {}

    print('\n── Label Propagation ──')
    sc, resid = run_lp(adj, labels_oh, tr_idx, va_idx, te_idx)
    q1_store[tag]['LP'] = {'scores': sc, 'residuals': resid}

    print('\n── Node2Vec ──')
    sc = run_n2v_planetoid(data, ds.num_classes)
    q1_store[tag]['Node2Vec'] = {'scores': sc}
    print(f"  mean={np.mean(sc):.4f}  std={np.std(sc):.4f}")

    print('\n── GCN ──')
    sc, tr_h, va_h = run_gcn_planetoid(adj, feats, labels_oh, tr_idx, va_idx, te_idx)
    q1_store[tag]['GCN'] = {'scores': sc, 'tr_hist': tr_h, 'va_hist': va_h}
    print(f"  mean={np.mean(sc):.4f}  std={np.std(sc):.4f}")

print(f"\n{'='*56}\nOGBN-PROTEINS  (ROC-AUC)\n{'='*56}")
q1_store['ogbn-proteins'] = {}

print('\n── Label Propagation ──')
sc, resid = run_lp_proteins(prot_data, prot_splits)
q1_store['ogbn-proteins']['LP'] = {'scores': sc, 'residuals': resid}

print('\n── Node2Vec ──')
sc = run_n2v_proteins(prot_data, prot_splits)
q1_store['ogbn-proteins']['Node2Vec'] = {'scores': sc}
print(f"  mean={np.mean(sc):.4f}  std={np.std(sc):.4f}")

print('\n── GCN ──')
sc, tr_h, va_h = run_gcn_proteins(prot_data, prot_splits)
q1_store['ogbn-proteins']['GCN'] = {'scores': sc, 'tr_hist': tr_h, 'va_hist': va_h}
print(f"  mean={np.mean(sc):.4f}  std={np.std(sc):.4f}")

In [ ]:
# ── Q1 Training Curves ────────────────────────────────────────────────────────
# One 2-panel figure per dataset.
#   Left : LP Frobenius-residual convergence (log-scale)
#   Right: GCN train & val loss (mean ± std across 10 runs)
# Node2Vec training is handled internally by gensim; no per-epoch loss exposed.

def plot_q1_curves(tag, store, gcn_ylabel='NLL Loss'):
    fig, (ax_lp, ax_gcn) = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(f'{tag.upper()} — Q1 Training Curves', fontweight='bold')

    resid = store[tag]['LP']['residuals']
    ax_lp.semilogy(range(1, len(resid) + 1), resid, color='steelblue', lw=2)
    ax_lp.set_xlabel('Iteration');  ax_lp.set_ylabel('Frobenius Residual (log)')
    ax_lp.set_title('Label Propagation — Convergence');  ax_lp.grid(True, alpha=0.3)

    max_ep = max(len(h) for h in store[tag]['GCN']['tr_hist'])
    def pad_runs(lst):
        return np.array([np.pad(h, (0, max_ep - len(h)), constant_values=h[-1]) for h in lst])
    tr_arr = pad_runs(store[tag]['GCN']['tr_hist'])
    va_arr = pad_runs(store[tag]['GCN']['va_hist'])
    ep = range(1, max_ep + 1)
    for arr, col, lbl in [(tr_arr, 'forestgreen', 'train'), (va_arr, 'crimson', 'val')]:
        m, s = arr.mean(0), arr.std(0)
        ax_gcn.plot(ep, m, color=col, lw=2, label=f'{lbl} loss')
        ax_gcn.fill_between(ep, m - s, m + s, alpha=0.2, color=col)
    ax_gcn.set_xlabel('Epoch');  ax_gcn.set_ylabel(gcn_ylabel)
    ax_gcn.set_title('GCN — Train / Val Loss');  ax_gcn.legend(fontsize=9);  ax_gcn.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'q1_curves_{tag}.pdf', bbox_inches='tight');  plt.show()

for tag in ['cora', 'citeseer', 'pubmed']:
    plot_q1_curves(tag, q1_store)
plot_q1_curves('ogbn-proteins', q1_store, gcn_ylabel='BCE Loss')

In [ ]:
# ── Q1 Per-Dataset Tables ─────────────────────────────────────────────────────
algos_q1 = ['LP', 'Node2Vec', 'GCN']
for tag in ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']:
    metric = 'ROC-AUC' if tag == 'ogbn-proteins' else 'Accuracy'
    rows = [{'Run': r+1, **{a: f"{q1_store[tag][a]['scores'][r]:.4f}" for a in algos_q1}}
            for r in range(N_RUNS)]
    df = pd.DataFrame(rows).set_index('Run')
    df.loc['mean'] = {a: f"{np.mean(q1_store[tag][a]['scores']):.4f}" for a in algos_q1}
    df.loc['std']  = {a: f"{np.std(q1_store[tag][a]['scores']):.4f}"  for a in algos_q1}
    print(f"\n{'─'*50}\n {tag.upper()}  — Test {metric}\n{'─'*50}")
    print(df.to_string())

# Summary pivot
rows = []
for tag in ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']:
    m = 'ROC-AUC' if tag == 'ogbn-proteins' else 'Accuracy'
    for algo in algos_q1:
        v = q1_store[tag][algo]['scores']
        rows.append({'Dataset': tag, 'Algorithm': algo,
                     'Result': f"{np.mean(v):.4f} \u00b1 {np.std(v):.4f}"})
pivot = pd.DataFrame(rows).pivot_table(
    values='Result', index='Dataset', columns='Algorithm', aggfunc='first')[algos_q1]
print("\n" + "="*65)
print("Q1 — Node Classification  (mean \u00b1 std, 10 runs)")
print("="*65);  print(pivot.to_string())

# Bar plot
tags_q1  = ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']
titles_q1 = ['Cora\n(Accuracy)', 'CiteSeer\n(Accuracy)',
              'PubMed\n(Accuracy)', 'ogbn-proteins\n(ROC-AUC)']
bar_cols  = ['steelblue', 'darkorange', 'seagreen']
fig, axes = plt.subplots(1, 4, figsize=(17, 5))
for ax, tag, title in zip(axes, tags_q1, titles_q1):
    means = [np.mean(q1_store[tag][a]['scores']) for a in algos_q1]
    stds  = [np.std(q1_store[tag][a]['scores'])  for a in algos_q1]
    bars  = ax.bar(algos_q1, means, yerr=stds, capsize=7,
                   color=bar_cols, alpha=0.82, edgecolor='k', lw=0.7)
    ax.set_title(title, fontsize=10);  ax.set_ylabel('Score')
    ax.set_ylim(0, min(1.15, max(means) + max(stds) + 0.15));  ax.grid(axis='y', alpha=0.3)
    for bar, m, s in zip(bars, means, stds):
        ax.text(bar.get_x() + bar.get_width()/2, m+s+0.01, f'{m:.3f}',
                ha='center', va='bottom', fontsize=8)
fig.suptitle('Q1 — Node Classification: Algorithm Comparison', fontsize=13, fontweight='bold')
plt.tight_layout();  plt.savefig('q1_comparison.pdf', bbox_inches='tight');  plt.show()

---## Q2 — Link Prediction**Protocol**: for each run, drop 20% of edges uniformly at random as the held-out positive set.  **Negative sampling**: sample an equal number of random non-edges; the same set is used by all algorithms within a run (consistent).  **Metric**: AUC (mean ± std over 10 runs).| Algorithm | Features | Notes ||-----------|----------|-------|| **Common Neighbours** | none | structural heuristic || **Jaccard Coefficient** | none | structural heuristic || **GCN encoder-decoder** | node features | inner-product edge scorer |

In [ ]:
# ── Q2 Edge Splitting & Negative Sampling ─────────────────────────────────────
# Based on Q2.py: randomly drop 20% of edges; sample equal negative edges.
# Negative sampling strategy: uniform random non-edges (no self-loops,
# not in the full edge set). Same negatives shared across all algorithms
# within a run for fair comparison.

def split_edges(edge_index: torch.Tensor, n_nodes: int,
                held_out_frac=0.2, seed=0):
    rng    = np.random.RandomState(seed)
    edges  = edge_index.T.numpy()           # (E, 2)
    n_test = int(len(edges) * held_out_frac)
    perm   = rng.permutation(len(edges))
    pos_test  = edges[perm[:n_test]]
    train_arr = edges[perm[n_test:]]
    train_ei  = torch.from_numpy(train_arr.T).long()

    full_set = set(map(tuple, edges))
    neg, tries = [], 0
    while len(neg) < n_test and tries < n_test * 30:
        u, v = rng.randint(0, n_nodes, 2)
        if u != v and (u,v) not in full_set and (v,u) not in full_set:
            neg.append([u, v]);  full_set.add((u, v))
        tries += 1
    return train_ei, pos_test, np.array(neg[:n_test])


def make_nx(edge_arr, n_nodes):
    G = nx.Graph()
    G.add_nodes_from(range(n_nodes))
    G.add_edges_from(map(tuple, edge_arr))
    return G

In [ ]:
# ── Structural Heuristic Link Predictors ──────────────────────────────────────
# Based on Q2.py logic. No features used.

def count_shared_nbrs(train_nx, pairs):
    """Common-neighbour count for each (u, v) pair."""
    return np.array([
        len(list(nx.common_neighbors(train_nx, int(u), int(v))))
        if train_nx.has_node(int(u)) and train_nx.has_node(int(v)) else 0
        for u, v in pairs
    ], dtype=float)


def jaccard_scores(train_nx, pairs):
    """Jaccard coefficient for each (u, v) pair."""
    out = []
    for u, v in pairs:
        u, v = int(u), int(v)
        if train_nx.has_node(u) and train_nx.has_node(v):
            out.append(next(nx.jaccard_coefficient(train_nx, [(u, v)]))[2])
        else:
            out.append(0.0)
    return np.array(out)


def eval_link_heuristic(score_fn, train_nx, pos_test, neg_test):
    all_pairs  = np.concatenate([pos_test, neg_test], axis=0)
    all_labels = np.concatenate([np.ones(len(pos_test)), np.zeros(len(neg_test))])
    return roc_auc_score(all_labels, score_fn(train_nx, all_pairs))

In [ ]:
# ── GCN Link Predictor (encoder-decoder) ──────────────────────────────────────
# Encoder: 2-layer GCN (same SpectralConvBlock) → node embeddings
# Decoder: σ(z_u · z_v)  (inner-product + sigmoid)
# Training: BCE on positive (train edges) + negative samples

class LinkEncoderGCN(nn.Module):
    def __init__(self, n_feat, n_hid, n_emb, p_drop=0.5):
        super().__init__()
        self.enc1   = SpectralConvBlock(n_feat, n_hid, p_drop)
        self.enc2   = nn.Linear(n_hid, n_emb)
        self.p_drop = p_drop

    def encode(self, x, norm_adj):
        h = self.enc1(x, norm_adj)
        h = F.dropout(h, p=self.p_drop, training=self.training)
        return self.enc2(h)

    @staticmethod
    def score_pairs(z, node_pairs):
        src, dst = node_pairs[:, 0], node_pairs[:, 1]
        return torch.sigmoid((z[src] * z[dst]).sum(dim=1))


def train_link_gcn(adj_train, feats, pos_tr, neg_tr, pos_te, neg_te,
                   n_hid=64, n_emb=32, epochs=100, lr=1e-2, p_drop=0.5, seed=0):
    fix_seed(seed)
    adj_t  = scipy_to_sparse_torch(row_normalise(adj_train))
    feat_t = torch.FloatTensor(feats)

    def make_batch(pos, neg):
        pairs  = torch.LongTensor(np.concatenate([pos, neg]))
        labels = torch.FloatTensor(np.concatenate([np.ones(len(pos)), np.zeros(len(neg))]))
        return pairs, labels

    tr_pairs, tr_lbl = make_batch(pos_tr, neg_tr)
    va_pairs, va_lbl = make_batch(pos_te, neg_te)

    net = LinkEncoderGCN(feats.shape[1], n_hid, n_emb, p_drop)
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    tr_hist, va_hist = [], []

    for _ in range(epochs):
        net.train();  opt.zero_grad()
        z    = net.encode(feat_t, adj_t)
        loss = F.binary_cross_entropy(net.score_pairs(z, tr_pairs), tr_lbl)
        loss.backward();  opt.step()
        net.eval()
        with torch.no_grad():
            z      = net.encode(feat_t, adj_t)
            va_loss = F.binary_cross_entropy(net.score_pairs(z, va_pairs), va_lbl).item()
        tr_hist.append(loss.item());  va_hist.append(va_loss)

    net.eval()
    with torch.no_grad():
        z     = net.encode(feat_t, adj_t)
        pairs = torch.LongTensor(np.concatenate([pos_te, neg_te]))
        probs = net.score_pairs(z, pairs).numpy()
    true = np.concatenate([np.ones(len(pos_te)), np.zeros(len(neg_te))])
    return roc_auc_score(true, probs), tr_hist, va_hist


def run_gcn_lp(data, n_runs=N_RUNS):
    feats = data.x.numpy().astype(np.float32)
    aucs, all_tr, all_va = [], [], []
    for run in range(n_runs):
        train_ei, pos_te, neg_te = split_edges(data.edge_index, data.num_nodes, seed=run)
        adj_tr = build_adj(train_ei, data.num_nodes)
        tr_pos = train_ei.T.numpy()
        rng    = np.random.RandomState(run)
        neg_tr = neg_te[rng.choice(len(neg_te), min(len(tr_pos), len(neg_te)), replace=False)]

        auc, tr_h, va_h = train_link_gcn(adj_tr, feats, tr_pos, neg_tr, pos_te, neg_te, seed=run*17)
        aucs.append(auc);  all_tr.append(tr_h);  all_va.append(va_h)
        print(f"  GCN-LP run {run+1:02d}/{n_runs}: auc={auc:.4f}")
    return aucs, all_tr, all_va

In [ ]:
# ── Q2 Experiments ────────────────────────────────────────────────────────────
q2_store = {}

for tag in ['cora', 'citeseer', 'pubmed']:
    data = pyg_catalog[tag][0]
    print(f"\n{'='*56}\n{tag.upper()} — Link Prediction\n{'='*56}")
    q2_store[tag] = {a: {'aucs': []} for a in ['Common Nbrs', 'Jaccard', 'GCN']}
    for k in ['GCN']:  q2_store[tag][k].update({'tr_hist': [], 'va_hist': []})

    for run in range(N_RUNS):
        train_ei, pos_te, neg_te = split_edges(data.edge_index, data.num_nodes, seed=run)
        train_nx = make_nx(train_ei.T.numpy(), data.num_nodes)
        q2_store[tag]['Common Nbrs']['aucs'].append(
            eval_link_heuristic(count_shared_nbrs, train_nx, pos_te, neg_te))
        q2_store[tag]['Jaccard']['aucs'].append(
            eval_link_heuristic(jaccard_scores, train_nx, pos_te, neg_te))
        cn  = q2_store[tag]['Common Nbrs']['aucs'][-1]
        jac = q2_store[tag]['Jaccard']['aucs'][-1]
        print(f"  run {run+1:02d}/{N_RUNS}: CN={cn:.4f}  Jaccard={jac:.4f}")

    print('\n── GCN link predictor ──')
    aucs, tr_h, va_h = run_gcn_lp(data)
    q2_store[tag]['GCN'].update({'aucs': aucs, 'tr_hist': tr_h, 'va_hist': va_h})
    for algo in ['Common Nbrs', 'Jaccard', 'GCN']:
        v = q2_store[tag][algo]['aucs']
        print(f"  {algo:<14} mean={np.mean(v):.4f}  std={np.std(v):.4f}")

# ogbn-proteins: subsampled to 5 000 nodes (heuristic tractability)
print(f"\n{'='*56}\nOGBN-PROTEINS — Link Prediction (5 000-node subgraph)\n{'='*56}")
fix_seed(0)
sub_n     = 5000
sub_nodes = np.random.choice(prot_data.num_nodes, sub_n, replace=False)
nmap      = {int(o): i for i, o in enumerate(sub_nodes)}
ei_np     = prot_data.edge_index.T.numpy()
mask      = np.isin(ei_np[:,0], sub_nodes) & np.isin(ei_np[:,1], sub_nodes)
sub_e     = np.array([[nmap[int(u)], nmap[int(v)]] for u, v in ei_np[mask]])
sub_ei    = torch.from_numpy(sub_e.T).long()
sub_data  = PyGData(x=prot_data.x[sub_nodes], edge_index=sub_ei,
                    y=prot_data.y[sub_nodes], num_nodes=sub_n)
print(f"Subgraph: {sub_n} nodes, {sub_ei.size(1)} edges")

q2_store['ogbn-proteins'] = {a: {'aucs': []} for a in ['Common Nbrs', 'Jaccard', 'GCN']}
for k in ['GCN']:  q2_store['ogbn-proteins'][k].update({'tr_hist': [], 'va_hist': []})
for run in range(N_RUNS):
    train_ei, pos_te, neg_te = split_edges(sub_ei, sub_n, seed=run)
    train_nx = make_nx(train_ei.T.numpy(), sub_n)
    q2_store['ogbn-proteins']['Common Nbrs']['aucs'].append(
        eval_link_heuristic(count_shared_nbrs, train_nx, pos_te, neg_te))
    q2_store['ogbn-proteins']['Jaccard']['aucs'].append(
        eval_link_heuristic(jaccard_scores, train_nx, pos_te, neg_te))
    cn  = q2_store['ogbn-proteins']['Common Nbrs']['aucs'][-1]
    jac = q2_store['ogbn-proteins']['Jaccard']['aucs'][-1]
    print(f"  run {run+1:02d}/{N_RUNS}: CN={cn:.4f}  Jaccard={jac:.4f}")

print('\n── GCN link predictor ──')
aucs, tr_h, va_h = run_gcn_lp(sub_data)
q2_store['ogbn-proteins']['GCN'].update({'aucs': aucs, 'tr_hist': tr_h, 'va_hist': va_h})
for algo in ['Common Nbrs', 'Jaccard', 'GCN']:
    v = q2_store['ogbn-proteins'][algo]['aucs']
    print(f"  {algo:<14} mean={np.mean(v):.4f}  std={np.std(v):.4f}")

In [ ]:
# ── Q2 GCN Training Curves ────────────────────────────────────────────────────
# One figure per dataset: GCN link-predictor train/val BCE loss.
# Common Neighbours and Jaccard have no iterative training phase.

def plot_q2_curves(tag, store):
    fig, ax = plt.subplots(figsize=(7, 4))
    fig.suptitle(f'{tag.upper()} — Q2 GCN Link Predictor: Train/Val Loss',
                 fontweight='bold')
    max_ep = max(len(h) for h in store[tag]['GCN']['tr_hist'])
    def pad(lst):
        return np.array([np.pad(h, (0, max_ep-len(h)), constant_values=h[-1]) for h in lst])
    tr_arr = pad(store[tag]['GCN']['tr_hist'])
    va_arr = pad(store[tag]['GCN']['va_hist'])
    ep = range(1, max_ep + 1)
    for arr, col, lbl in [(tr_arr, 'teal', 'train'), (va_arr, 'tomato', 'val')]:
        m, s = arr.mean(0), arr.std(0)
        ax.plot(ep, m, color=col, lw=2, label=f'{lbl} loss')
        ax.fill_between(ep, m-s, m+s, alpha=0.2, color=col)
    ax.set_xlabel('Epoch');  ax.set_ylabel('BCE Loss')
    ax.legend(fontsize=9);   ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'q2_gcn_curves_{tag}.pdf', bbox_inches='tight');  plt.show()

for tag in ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']:
    plot_q2_curves(tag, q2_store)

In [ ]:
# ── Q2 Per-Dataset Tables ─────────────────────────────────────────────────────
algos_q2 = ['Common Nbrs', 'Jaccard', 'GCN']
for tag in ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']:
    rows = [{'Run': r+1, **{a: f"{q2_store[tag][a]['aucs'][r]:.4f}" for a in algos_q2}}
            for r in range(N_RUNS)]
    df = pd.DataFrame(rows).set_index('Run')
    df.loc['mean'] = {a: f"{np.mean(q2_store[tag][a]['aucs']):.4f}" for a in algos_q2}
    df.loc['std']  = {a: f"{np.std(q2_store[tag][a]['aucs']):.4f}"  for a in algos_q2}
    print(f"\n{'─'*50}\n {tag.upper()}  — Test AUC\n{'─'*50}")
    print(df.to_string())

rows = []
for tag in ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']:
    for algo in algos_q2:
        v = q2_store[tag][algo]['aucs']
        rows.append({'Dataset': tag, 'Algorithm': algo,
                     'AUC': f"{np.mean(v):.4f} \u00b1 {np.std(v):.4f}"})
pivot = pd.DataFrame(rows).pivot_table(
    values='AUC', index='Dataset', columns='Algorithm', aggfunc='first')[algos_q2]
print("\n"+"="*65)
print("Q2 — Link Prediction  (mean \u00b1 std, 10 runs)")
print("="*65);  print(pivot.to_string())

tags_q2   = ['cora', 'citeseer', 'pubmed', 'ogbn-proteins']
titles_q2 = ['Cora\n(AUC)', 'CiteSeer\n(AUC)', 'PubMed\n(AUC)', 'ogbn-proteins\n(AUC)']
bar_cols  = ['steelblue', 'darkorange', 'seagreen']
fig, axes = plt.subplots(1, 4, figsize=(17, 5))
for ax, tag, title in zip(axes, tags_q2, titles_q2):
    means = [np.mean(q2_store[tag][a]['aucs']) for a in algos_q2]
    stds  = [np.std(q2_store[tag][a]['aucs'])  for a in algos_q2]
    bars  = ax.bar(algos_q2, means, yerr=stds, capsize=7,
                   color=bar_cols, alpha=0.82, edgecolor='k', lw=0.7)
    ax.set_title(title, fontsize=10);  ax.set_ylabel('AUC')
    ax.set_ylim(0, min(1.15, max(means)+max(stds)+0.15));  ax.grid(axis='y', alpha=0.3)
    ax.set_xticks(range(len(algos_q2)))
    ax.set_xticklabels(['Com.Nbrs', 'Jaccard', 'GCN'], fontsize=8)
    for bar, m, s in zip(bars, means, stds):
        ax.text(bar.get_x()+bar.get_width()/2, m+s+0.01, f'{m:.3f}',
                ha='center', va='bottom', fontsize=8)
fig.suptitle('Q2 — Link Prediction: Algorithm Comparison', fontsize=13, fontweight='bold')
plt.tight_layout();  plt.savefig('q2_comparison.pdf', bbox_inches='tight');  plt.show()